In [4]:
import pandas as pd
import numpy as np
import pickle
import os

import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
data = pd.read_csv("../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [7]:
texts = data.loc[0, "sentences"]
texts

['sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı',
 'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi',
 'polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı',
 'rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi',
 'polisin çalışması devam ediyor']

In [19]:
texts_all = " [SEP] ".join(texts)
texts_all

'sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı [SEP] ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi [SEP] polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı [SEP] rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi [SEP] polisin çalışması devam ediyor'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

In [12]:
tokenizer

PreTrainedTokenizerFast(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [21]:
tokenizer([texts_all], truncation=True, padding=True, max_length=100*10)

{'input_ids': [[2, 8449, 24635, 16217, 2446, 12088, 58520, 2676, 40082, 3574, 2095, 2058, 6512, 78175, 22704, 3149, 15279, 2189, 16217, 2932, 5735, 2065, 3149, 12874, 2789, 4190, 3730, 12305, 2386, 54080, 2956, 31083, 12088, 3, 2683, 27631, 116288, 1019, 115973, 2280, 15279, 1011, 84087, 2023, 8546, 7641, 8059, 4190, 2426, 18517, 41569, 1971, 3, 11821, 5267, 29577, 53677, 18387, 2120, 4190, 21783, 2282, 52565, 4763, 12088, 3, 64557, 4190, 16853, 11649, 119506, 2537, 15279, 1011, 84087, 5326, 4673, 3985, 4341, 3, 11821, 52565, 2094, 2524, 2992, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
tokenizer(texts, truncation=True, padding=True)["input_ids"]

[[2,
  8449,
  24635,
  16217,
  2446,
  12088,
  58520,
  2676,
  40082,
  3574,
  2095,
  2058,
  6512,
  78175,
  22704,
  3149,
  15279,
  2189,
  16217,
  2932,
  5735,
  2065,
  3149,
  12874,
  2789,
  4190,
  3730,
  12305,
  2386,
  54080,
  2956,
  31083,
  12088,
  3],
 [2,
  2683,
  27631,
  116288,
  1019,
  115973,
  2280,
  15279,
  1011,
  84087,
  2023,
  8546,
  7641,
  8059,
  4190,
  2426,
  18517,
  41569,
  1971,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [2,
  11821,
  5267,
  29577,
  53677,
  18387,
  2120,
  4190,
  21783,
  2282,
  52565,
  4763,
  12088,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [2,
  64557,
  4190,
  16853,
  11649,
  119506,
  2537,
  15279,
  1011,
  84087,
  5326,
  4673,
  3985,
  4341,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [2,
  11821,
  52565,
  2094,
  2524,
  2992,
  3,
  

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})